In [1]:
!nvidia-smi

Sun May  4 12:10:35 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   77C    P0             34W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
!pip install transformers[sentencepiece] datasets sacrebleu rouge_score py7zr -q

In [3]:
!pip install --upgrade accelerate
!pip uninstall -y transformers accelerate
!pip install transformers accelerate

Found existing installation: transformers 4.51.3
Uninstalling transformers-4.51.3:
  Successfully uninstalled transformers-4.51.3
Found existing installation: accelerate 1.6.0
Uninstalling accelerate-1.6.0:
  Successfully uninstalled accelerate-1.6.0
  Using cached transformers-4.51.3-py3-none-any.whl.metadata (38 kB)
  Using cached accelerate-1.6.0-py3-none-any.whl.metadata (19 kB)
Using cached transformers-4.51.3-py3-none-any.whl (10.4 MB)
Using cached accelerate-1.6.0-py3-none-any.whl (354 kB)


In [4]:
!pip install evaluate


In [5]:
from transformers import pipeline, set_seed
from datasets import load_dataset, load_from_disk
import matplotlib.pyplot as plt
from datasets import load_dataset
import pandas as pd
import evaluate


from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

import nltk
from nltk.tokenize import sent_tokenize

from tqdm import tqdm
import torch

nltk.download("punkt")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [6]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [7]:
model_ckpt = "google/pegasus-cnn_dailymail"

tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
#dowload & unzip data

!wget https://github.com/arabinda0704/Text-Summarizer/blob/main/summarizer-data.zip
!unzip summarizer-data.zip

--2025-05-04 12:11:45--  https://github.com/arabinda0704/Text-Summarizer/blob/main/summarizer-data.zip
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘summarizer-data.zip.3’

summarizer-data.zip     [  <=>               ] 166.56K   689KB/s    in 0.2s    

2025-05-04 12:11:45 (689 KB/s) - ‘summarizer-data.zip.3’ saved [170561]

Archive:  summarizer-data.zip
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of summarizer-data.zip or
        summarizer-data.zip.zip, and cannot find summarizer-data.zip.ZIP, period.


In [9]:
from datasets import load_dataset

dataset_samsum = load_dataset("samsum")


In [10]:
dataset_samsum.save_to_disk("samsum_dataset")


Saving the dataset (0/1 shards):   0%|          | 0/14732 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/819 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/818 [00:00<?, ? examples/s]

In [11]:
from datasets import load_from_disk

dataset_samsum = load_from_disk("samsum_dataset")

In [12]:
split_lengths = [len(dataset_samsum[split])for split in dataset_samsum]

print(f"Split lengths: {split_lengths}")
print(f"Features: {dataset_samsum['train'].column_names}")
print("\nDialogue:")

print(dataset_samsum["test"][1]["dialogue"])

print("\nSummary:")

print(dataset_samsum["test"][1]["summary"])

Split lengths: [14732, 819, 818]
Features: ['id', 'dialogue', 'summary']

Dialogue:
Eric: MACHINE!
Rob: That's so gr8!
Eric: I know! And shows how Americans see Russian ;)
Rob: And it's really funny!
Eric: I know! I especially like the train part!
Rob: Hahaha! No one talks to the machine like that!
Eric: Is this his only stand-up?
Rob: Idk. I'll check.
Eric: Sure.
Rob: Turns out no! There are some of his stand-ups on youtube.
Eric: Gr8! I'll watch them now!
Rob: Me too!
Eric: MACHINE!
Rob: MACHINE!
Eric: TTYL?
Rob: Sure :)

Summary:
Eric and Rob are going to watch a stand-up on youtube.


In [13]:
def convert_examples_to_features(example_batch):
    input_encodings = tokenizer(example_batch['dialogue'] , max_length = 1024, truncation = True )

    with tokenizer.as_target_tokenizer():
        target_encodings = tokenizer(example_batch['summary'], max_length = 128, truncation = True )

    return {
        'input_ids' : input_encodings['input_ids'],
        'attention_mask': input_encodings['attention_mask'],
        'labels': target_encodings['input_ids']
    }


In [14]:
dataset_samsum_pt = dataset_samsum.map(convert_examples_to_features, batched = True)

In [15]:
dataset_samsum_pt["train"]

Dataset({
    features: ['id', 'dialogue', 'summary', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 14732
})

In [16]:
# Training

from transformers import DataCollatorForSeq2Seq

seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_pegasus)

In [17]:
!pip install --upgrade transformers


In [18]:
!pip install -U transformers


In [19]:
import transformers
print(transformers.__version__)

4.51.3


In [20]:
!ls

pegasus-samsum	      samsum_dataset	     summarizer-data.zip.2  wandb
pegasus-samsum-model  summarizer-data.zip    summarizer-data.zip.3
sample_data	      summarizer-data.zip.1  tokenizer


In [21]:
import transformers
print(transformers.__file__)


/usr/local/lib/python3.11/dist-packages/transformers/__init__.py


In [22]:
# !pip uninstall -y transformers datasets evaluate


In [23]:
!pip install -U transformers datasets evaluate


In [24]:
# !pip uninstall -y transformers datasets evaluate
# !pip install -U transformers datasets evaluate

In [25]:
# from transformers import TrainingArguments

# trainer_args = TrainingArguments(
#     output_dir='pegasus-samsum',
#     num_train_epochs=1,
#     warmup_steps=500,
#     per_device_train_batch_size=1,
#     per_device_eval_batch_size=1,
#     weight_decay=0.01,
#     logging_steps=10,
#     evaluation_strategy='steps',
#     eval_steps=500,
#     save_steps=int(1e6),
#     gradient_accumulation_steps=16
# )
from transformers import TrainingArguments

trainer_args = TrainingArguments(
    output_dir='pegasus-samsum',
    num_train_epochs=1, #3 for better result
    warmup_steps=500,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    weight_decay=0.01,
    logging_steps=10,
    # Remove evaluation_strategy temporarily
    eval_steps=500,
    save_steps=int(1e6),
    gradient_accumulation_steps=16
)


In [26]:
from transformers import Trainer

trainer = Trainer(model=model_pegasus, args=trainer_args,
                  tokenizer=tokenizer, data_collator=seq2seq_data_collator,
                  train_dataset=dataset_samsum_pt["train"],
                  eval_dataset=dataset_samsum_pt["validation"])

<ipython-input-26-7308b86384ca>:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(model=model_pegasus, args=trainer_args,


In [ ]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Currently logged in as: arabinda0704 (arabinda0704-jadavpur-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
10,3.217100
20,2.993600
30,3.108200
40,3.021200
50,2.675300
60,2.690500
70,2.725900
80,2.526900
90,2.375400
100,2.508000


In [ ]:
# Evaluation

def generate_batch_sized_chunks(list_of_elements, batch_size):
    """split the dataset into smaller batches that we can process simultaneously
    Yield successive batch-sized chunks from list_of_elements."""
    for i in range(0, len(list_of_elements), batch_size):
        yield list_of_elements[i : i + batch_size]



def calculate_metric_on_test_ds(dataset, metric, model, tokenizer,
                               batch_size=16, device=device,
                               column_text="article",
                               column_summary="highlights"):
    article_batches = list(generate_batch_sized_chunks(dataset[column_text], batch_size))
    target_batches = list(generate_batch_sized_chunks(dataset[column_summary], batch_size))

    for article_batch, target_batch in tqdm(
        zip(article_batches, target_batches), total=len(article_batches)):

        inputs = tokenizer(article_batch, max_length=1024,  truncation=True,
                        padding="max_length", return_tensors="pt")

        summaries = model.generate(input_ids=inputs["input_ids"].to(device),
                         attention_mask=inputs["attention_mask"].to(device),
                         length_penalty=0.8, num_beams=8, max_length=128)
        ''' parameter for length penalty ensures that the model does not generate sequences that are too long. '''

        # Finally, we decode the generated texts,
        # replace the  token, and add the decoded texts with the references to the metric.
        decoded_summaries = [tokenizer.decode(s, skip_special_tokens=True,
                                clean_up_tokenization_spaces=True)
               for s in summaries]

        decoded_summaries = [d.replace("", " ") for d in decoded_summaries]


        metric.add_batch(predictions=decoded_summaries, references=target_batch)

    #  Finally compute and return the ROUGE scores.
    score = metric.compute()
    return score

In [ ]:
rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]
rouge_metric = evaluate.load("rouge")

In [ ]:
print(len(dataset_samsum['train']))  # Make sure it's greater than 0


In [ ]:
print(dataset_samsum['train'].column_names)  # Make sure 'dialogue' and 'summary' are valid columns


In [ ]:
print(dataset_samsum['train'][0:5])  # Check first 5 examples


In [ ]:
import torch
import evaluate
import pandas as pd

# Load the Rouge metric using the evaluate library
rouge_metric = evaluate.load("rouge")

# Define the function to calculate the Rouge scores on the dataset
def calculate_metric_on_test_ds(dataset, metric, model, tokenizer, batch_size=2, column_text='dialogue', column_summary='summary'):
    # Ensure the model is on the correct device (CUDA or CPU)
    device = model.device if next(model.parameters()).is_cuda else torch.device('cpu')

    # Make sure the dataset is not empty and handle slicing
    assert len(dataset) > 0, "Dataset is empty!"

    # Convert the dataset to a list of examples
    dataset_list = [{"dialogue": dataset['dialogue'][i], "summary": dataset['summary'][i]} for i in range(len(dataset['dialogue']))]

    # Create containers for predictions and references
    predictions = []
    references = []

    # Iterate over the dataset in batches
    for i in range(0, len(dataset_list), batch_size):
        # Get a batch by slicing directly from the list
        batch = dataset_list[i:i + batch_size]

        texts = [item[column_text] for item in batch]
        summaries = [item[column_summary] for item in batch]

        # Tokenize the batch (ensure truncation and padding)
        inputs = tokenizer(texts, return_tensors="pt", padding=True, truncation=True, max_length=512)

        # Move the inputs to the correct device
        inputs = {key: value.to(device) for key, value in inputs.items()}

        # Generate model predictions (this part depends on your model type)
        with torch.no_grad():
            outputs = model.generate(inputs['input_ids'], max_length=512, num_beams=4, early_stopping=True)

        # Decode the generated outputs to get the predictions
        predictions.extend([tokenizer.decode(output, skip_special_tokens=True) for output in outputs])
        references.extend(summaries)

    # Calculate the Rouge scores using the metric
    result = metric.compute(predictions=predictions, references=references)
    return result

# Ensure that dataset_samsum['test'] has at least 10 samples
test_data = dataset_samsum['test'][:min(10, len(dataset_samsum['test']))]

# Calculate Rouge scores for a subset of the test dataset
score = calculate_metric_on_test_ds(
    test_data,
    rouge_metric,
    trainer.model,
    tokenizer,
    batch_size=2,
    column_text='dialogue',
    column_summary='summary'
)

# Create a dictionary to store the Rouge scores for each metric (e.g., rouge1, rouge2, rougeL)
rouge_dict = {rn: score[rn] for rn in score}

# Convert the result into a DataFrame for easier viewing
rouge_df = pd.DataFrame(rouge_dict, index=[f'pegasus'])

# Display the scores
print(rouge_df)


In [ ]:
## Save model
model_pegasus.save_pretrained("pegasus-samsum-model")

In [ ]:
## Save tokenizer
tokenizer.save_pretrained("tokenizer")

In [ ]:
#Load

tokenizer = AutoTokenizer.from_pretrained("/content/tokenizer")

In [ ]:
#Prediction

gen_kwargs = {"length_penalty": 0.8, "num_beams":8, "max_length": 128}



sample_text = dataset_samsum["train"][0]["dialogue"]

reference = dataset_samsum["train"][0]["summary"]

pipe = pipeline("summarization", model="pegasus-samsum-model",tokenizer=tokenizer)

##
print("Dialogue:")
print(sample_text)


print("\nReference Summary:")
print(reference)


print("\nModel Summary:")
print(pipe(sample_text, **gen_kwargs)[0]["summary_text"])